<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Validation/InceptionV3_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/Proper_dataset/train"
test_dir = "/content/drive/MyDrive/Proper_dataset/val"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), class_mode='binary')

Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load InceptionV3 without top layers
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)  # Binary classification

model_inception = Model(inputs=base_model.input, outputs=output)

# Compile model
model_inception.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Train model
model_inception.fit(train_data, validation_data=test_data, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8584 - loss: 0.5459

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 572s 7s/step - accuracy: 0.8588 - loss: 0.5435 - val_accuracy: 0.9217 - val_loss: 0.1773
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 443s 5s/step - accuracy: 0.9312 - loss: 0.1828 - val_accuracy: 0.9478 - val_loss: 0.1345
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 442s 5s/step - accuracy: 0.9464 - loss: 0.1321 - val_accuracy: 0.9478 - val_loss: 0.1240
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 435s 5s/step - accuracy: 0.9625 - loss: 0.1106 - val_accuracy: 0.9462 - val_loss: 0.1261
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 436s 5s/step - accuracy: 0.9719 - loss: 0.0773 - val_accuracy: 0.9416 - val_loss: 0.1212
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 435s 5s/step - accuracy: 0.9775 - loss: 0.0632 - val_accuracy: 0.9493 - val_loss: 0.1206
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 434s 5s/step - accuracy: 0.9773 - loss: 0.0565 - val_accuracy: 0.9493 - val_loss: 0.1248
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 434s 5s/step - accuracy: 0.9819 - loss: 0.0437 - val_accuracy: 0.9524 - val_loss: 0.

In [6]:
test_loss, test_accuracy = model_inception.evaluate(test_data)
print(f"InceptionV3 Test Accuracy: {test_accuracy * 100:.2f}%")

 5/21 ━━━━━━━━━━━━━━━━━━━━ 1:06 4s/step - accuracy: 0.9573 - loss: 0.1012

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 85s 4s/step - accuracy: 0.9510 - loss: 0.1298
InceptionV3 Test Accuracy: 95.24%


In [5]:
import cv2
import numpy as np

# Prediction function
def predict_image_inception(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model_inception.predict(img)[0][0]
    if prediction > 0.5:
        print("🔴 Non-Mouth Image Detected (InceptionV3)")
    else:
        print("✅ Mouth Image Detected (InceptionV3)")

# Example usage
predict_image_inception("/content/drive/MyDrive/test_notMouth_img.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
✅ Mouth Image Detected (InceptionV3)
